<a href="https://colab.research.google.com/github/dannynacker/strobe_entrainment_periodicity_MSc/blob/main/generative_rox.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The following will prompt the number of session steps, their durations, start and end frequency, duty cycle, and amplitude, as well as LEDs per oscillator, with the (alleged) capacity to turn off LEDs in other oscillators already turned on in another. Further, the user is offered the option of either a periodic wave or an aperiodic wave (with step durations jittered via drawing from a Poisson process), albeit the step size constraint of the decimal placing reduces the capacity for aperiodicity, especially at lower frequencies.

In [7]:
import numpy as np

def poisson_flash_times(target_start_freq, target_end_freq, duration, duty_cycle,
                        min_interval=0.025, max_interval_factor=3.0, tolerance=0.05, max_attempts=100000):
    """Generates an aperiodic Poisson-distributed sequence of flashes."""
    start_cycle_length = 1.0 / target_start_freq
    end_cycle_length = 1.0 / target_end_freq

    for _ in range(max_attempts):
        times = []
        time = 0.0
        while time < duration:
            progress = time / duration
            current_cycle_length = start_cycle_length + progress * (end_cycle_length - start_cycle_length)
            interval = np.random.exponential(current_cycle_length * 0.8)
            interval = min(interval, current_cycle_length * max_interval_factor)
            interval = max(interval, min_interval)
            on_duration = interval * (duty_cycle / 100)
            time += interval
            if time < duration:
                times.append((time, on_duration))

        effective_start_freq = len([t for t in times if t[0] < duration * 0.5]) / (duration * 0.5)
        effective_end_freq = len([t for t in times if t[0] >= duration * 0.5]) / (duration * 0.5)

        if (abs(effective_start_freq - target_start_freq) <= tolerance and
            abs(effective_end_freq - target_end_freq) <= tolerance):
            return times
    raise ValueError("Failed to generate an aperiodic sequence within constraints.")

def format_time(total_seconds):
    """Formats TIM as hh:mm:ss.s with leading zeros."""
    minutes = int(total_seconds // 60)
    seconds = int(total_seconds % 60)
    fraction = total_seconds - int(total_seconds)
    return f"00:{minutes:02}:{seconds:02}.{int(fraction * 10)}"

def format_stp(step_duration, wave_type, start_freq, end_freq, start_duty, end_duty,
               led_assignments, start_intensity, end_intensity):
    """Formats a single STP line."""
    blocks = []
    for osc in range(4):
        if any(led_assignments[osc]):
            block = f"{wave_type},{start_freq[osc]:.2f},{end_freq[osc]:.2f},{start_duty[osc]},{end_duty[osc]},{','.join(map(str, led_assignments[osc]))},{start_intensity[osc]},{end_intensity[osc]}"
        else:
            block = f"0,{start_freq[osc]:.2f},{end_freq[osc]:.2f},{start_duty[osc]},{end_duty[osc]},0,0,0,0,0,0"
        blocks.append(block)
    return f'STP"{max(step_duration, 0.1):.1f},{",".join(blocks)}"'

def get_led_assignments():
    """Prompts the user for LED assignments per oscillator."""
    led_config = []
    for osc in range(4):
        osc_leds = []
        print(f"Enter LED assignments for Oscillator {osc+1} (0 [off] or 1 [on]):")
        for led in range(4):
            value = int(input(f"  LED {led+1}: "))
            while value not in [0, 1]:
                print("  Please enter 0 or 1.")
                value = int(input(f"  LED {led+1}: "))
            osc_leds.append(value)
        led_config.append(osc_leds)
    return led_config

def adjust_led_assignments(led_config):
    """Ensures only one oscillator is active."""
    active_index = 0
    for i in range(4):
        if sum(led_config[i]) > 0:
            active_index = i
            break
    for i in range(4):
        if i != active_index:
            led_config[i] = [0, 0, 0, 0]
    return led_config

def normalize_output(output_lines):
    """Ensures strict formatting consistency with session manager output."""
    return [line.rstrip() + "\n" for line in output_lines]

def generate_strobe_sequence():
    """Main function to generate the strobe sequence."""
    num_steps = int(input("Enter the number of steps in the sequence: "))
    output = []
    total_elapsed_time = 0.0
    step_params = []

    for step in range(num_steps):
        print(f"Step {step + 1}:")
        duration = float(input("  Enter step duration (seconds): "))
        periodicity = input("  Enter periodicity (p=Periodic, a=Aperiodic): ").strip().lower()
        start_freq = float(input("  Enter start effective frequency (Hz): "))
        end_freq = float(input("  Enter end effective frequency (Hz): "))
        start_l = int(input("  Enter start luminance (0-100): "))
        end_l = int(input("  Enter end luminance (0-100): "))
        start_d = int(input("  Enter start duty cycle (1-99): "))
        end_d = int(input("  Enter end duty cycle (1-99): "))
        wave_type = int(input("  Enter wave type (0=Off, 1=Square, 2=Sine): "))

        led_config = get_led_assignments()
        led_config = adjust_led_assignments(led_config)

        step_params.append((duration, periodicity, start_freq, end_freq,
                            start_l, end_l, start_d, end_d, wave_type, led_config))

    total_duration = sum([step[0] for step in step_params])
    output.append(f'TIM"{format_time(total_duration).strip()}"')
    output.append(f'DUR"{total_duration:.1f}"'.strip())  # Ensure no trailing spaces

    for step in step_params:
        (duration, periodicity, start_freq, end_freq,
         start_l, end_l, start_d, end_d, wave_type, led_config) = step

        if periodicity == "p":
            flash_times = [(i * (1.0 / start_freq), (start_d / 100) * (1.0 / start_freq))
                           for i in range(int(duration * start_freq))]
        else:
            flash_times = poisson_flash_times(start_freq, end_freq, duration, start_d)

        previous_onset = total_elapsed_time
        num_flashes = len(flash_times)

        for i, (onset, on_duration) in enumerate(flash_times):
            step_duration = max(onset - previous_onset, 0.100)
            previous_onset += step_duration

            #**Linear Interpolation Across STPs**
            interp_freq = start_freq + (i / max(1, num_flashes - 1)) * (end_freq - start_freq)

            stp_line = format_stp(step_duration, wave_type, [interp_freq]*4, [interp_freq]*4,
                                  [start_d]*4, [end_d]*4, led_config, [start_l]*4, [end_l]*4)
            output.append(stp_line)

        total_elapsed_time += duration


    normalized_output = normalize_output(output)

    with open("strobe_sequence.txt", "w", encoding="utf-8", newline="") as f:
        f.write("\r\n".join(output))

    print("Generated strobe_sequence.txt")

generate_strobe_sequence()

Enter the number of steps in the sequence: 1
Step 1:
  Enter step duration (seconds): 10
  Enter periodicity (p=Periodic, a=Aperiodic): p
  Enter start effective frequency (Hz): 10
  Enter end effective frequency (Hz): 20
  Enter start luminance (0-100): 50
  Enter end luminance (0-100): 50
  Enter start duty cycle (1-99): 50
  Enter end duty cycle (1-99): 50
  Enter wave type (0=Off, 1=Square, 2=Sine): 1
Enter LED assignments for Oscillator 1 (0 [off] or 1 [on]):
  LED 1: 1
  LED 2: 1
  LED 3: 1
  LED 4: 1
Enter LED assignments for Oscillator 2 (0 [off] or 1 [on]):
  LED 1: 0
  LED 2: 0
  LED 3: 0
  LED 4: 0
Enter LED assignments for Oscillator 3 (0 [off] or 1 [on]):
  LED 1: 0
  LED 2: 0
  LED 3: 0
  LED 4: 0
Enter LED assignments for Oscillator 4 (0 [off] or 1 [on]):
  LED 1: 0
  LED 2: 0
  LED 3: 0
  LED 4: 0
Generated strobe_sequence.txt


Further (uh-oh), the following replaces the aperiodic Poisson-distribution interval calculation (which gets lumped into 100ms bins anyways) with a stochastic frequency modulation approach (i.e., noise-based jitter as opposed to phase-based jitter like Lionel's process).

In [9]:
import numpy as np

def generate_stochastic_frequencies(target_start_freq, target_end_freq, duration, jitter_range, step_size=0.1):
    """Generates a sequence of frequencies fluctuating stochastically within a defined jitter range."""
    num_steps = int(duration / step_size)
    time_progress = np.linspace(0, 1, num_steps)
    base_frequencies = target_start_freq + time_progress * (target_end_freq - target_start_freq)

    # Apply stochastic jitter in the defined range
    jitter = np.random.uniform(-jitter_range, jitter_range, num_steps)
    modulated_frequencies = base_frequencies + jitter

    # Ensure frequencies remain within valid bounds
    modulated_frequencies = np.clip(modulated_frequencies, 0.5, 200)  # Device limits

    return modulated_frequencies

def format_time(total_seconds):
    """Formats TIM as hh:mm:ss.s with leading zeros."""
    minutes = int(total_seconds // 60)
    seconds = int(total_seconds % 60)
    fraction = total_seconds - int(total_seconds)
    return f"00:{minutes:02}:{seconds:02}.{int(fraction * 10)}"

def format_stp(step_duration, wave_type, start_freq, end_freq, start_duty, end_duty,
               led_assignments, start_intensity, end_intensity):
    """Formats a single STP line."""
    blocks = []
    for osc in range(4):
        if any(led_assignments[osc]):
            block = f"{wave_type},{start_freq[osc]:.2f},{end_freq[osc]:.2f},{start_duty[osc]},{end_duty[osc]},{','.join(map(str, led_assignments[osc]))},{start_intensity[osc]},{end_intensity[osc]}"
        else:
            block = f"0,{start_freq[osc]:.2f},{end_freq[osc]:.2f},{start_duty[osc]},{end_duty[osc]},0,0,0,0,0,0"
        blocks.append(block)
    return f'STP"{max(step_duration, 0.1):.1f},{",".join(blocks)}"'

def get_led_assignments():
    """Prompts the user for LED assignments per oscillator."""
    led_config = []
    for osc in range(4):
        osc_leds = []
        print(f"Enter LED assignments for Oscillator {osc+1} (0 [off] or 1 [on]):")
        for led in range(4):
            value = int(input(f"  LED {led+1}: "))
            while value not in [0, 1]:
                print("  Please enter 0 or 1.")
                value = int(input(f"  LED {led+1}: "))
            osc_leds.append(value)
        led_config.append(osc_leds)
    return led_config

def adjust_led_assignments(led_config):
    """Ensures only one oscillator is active at a time."""
    active_index = 0
    for i in range(4):
        if sum(led_config[i]) > 0:
            active_index = i
            break
    for i in range(4):
        if i != active_index:
            led_config[i] = [0, 0, 0, 0]
    return led_config

def normalize_output(output_lines):
    """Ensures strict formatting consistency with session manager output."""
    return [line.rstrip() + "\n" for line in output_lines]

def generate_strobe_sequence():
    """Main function to generate the strobe sequence."""
    num_steps = int(input("Enter the number of steps in the sequence: "))
    output = []
    total_elapsed_time = 0.0
    step_params = []

    for step in range(num_steps):
        print(f"Step {step + 1}:")
        duration = float(input("  Enter step duration (seconds): "))
        periodicity = input("  Enter periodicity (p=Periodic, a=Aperiodic): ").strip().lower()
        start_freq = float(input("  Enter start effective frequency (Hz): "))
        end_freq = float(input("  Enter end effective frequency (Hz): "))
        jitter_range = float(input("  Enter jitter range (Hz): "))  # New input for stochastic jitter
        start_l = int(input("  Enter start luminance (0-100): "))
        end_l = int(input("  Enter end luminance (0-100): "))
        start_d = int(input("  Enter start duty cycle (1-99): "))
        end_d = int(input("  Enter end duty cycle (1-99): "))
        wave_type = int(input("  Enter wave type (0=Off, 1=Square, 2=Sine): "))
        led_config = get_led_assignments()
        led_config = adjust_led_assignments(led_config)

        step_params.append((duration, periodicity, start_freq, end_freq, jitter_range,
                            start_l, end_l, start_d, end_d, wave_type, led_config))

    total_duration = sum([step[0] for step in step_params])
    output.append(f'TIM"{format_time(total_duration).strip()}"')
    output.append(f'DUR"{total_duration:.1f}"'.strip())

    for step in step_params:
        (duration, periodicity, start_freq, end_freq, jitter_range,
         start_l, end_l, start_d, end_d, wave_type, led_config) = step

        num_intervals = int(duration / 0.1)  # Number of STPs in this step

        if periodicity == "p":
            # 🔹 Interpolate smoothly from start_freq to end_freq
            modulated_frequencies = np.linspace(start_freq, end_freq, num_intervals)
        else:
            # 🔹 Generate stochastic frequencies with jitter
            modulated_frequencies = generate_stochastic_frequencies(start_freq, end_freq, duration, jitter_range)

        for i, freq in enumerate(modulated_frequencies):
            # Ensure the interpolated frequency is properly used
            stp_line = format_stp(0.1, wave_type, [freq]*4, [freq]*4, [start_d]*4, [end_d]*4, led_config, [start_l]*4, [end_l]*4)
            output.append(stp_line)

        for i, freq in enumerate(modulated_frequencies):
            stp_line = format_stp(0.1, wave_type, [freq]*4, [freq]*4, [start_d]*4, [end_d]*4, led_config, [start_l]*4, [end_l]*4)
            output.append(stp_line)

    with open("strobe_sequence.txt", "w", encoding="utf-8", newline="") as f:
        f.write("\r\n".join(output))

    print("Generated strobe_sequence.txt.")

generate_strobe_sequence()

Enter the number of steps in the sequence: 1
Step 1:
  Enter step duration (seconds): 10
  Enter periodicity (p=Periodic, a=Aperiodic): p
  Enter start effective frequency (Hz): 10
  Enter end effective frequency (Hz): 20
  Enter jitter range (Hz): 2
  Enter start luminance (0-100): 50
  Enter end luminance (0-100): 50
  Enter start duty cycle (1-99): 50
  Enter end duty cycle (1-99): 50
  Enter wave type (0=Off, 1=Square, 2=Sine): 1
Enter LED assignments for Oscillator 1 (0 [off] or 1 [on]):
  LED 1: 1
  LED 2: 1
  LED 3: 1
  LED 4: 1
Enter LED assignments for Oscillator 2 (0 [off] or 1 [on]):
  LED 1: 1
  LED 2: 1
  LED 3: 1
  LED 4: 1
Enter LED assignments for Oscillator 3 (0 [off] or 1 [on]):
  LED 1: 1
  LED 2: 1
  LED 3: 1
  LED 4: 1
Enter LED assignments for Oscillator 4 (0 [off] or 1 [on]):
  LED 1: 1
  LED 2: 1
  LED 3: 1
  LED 4: 1
Generated strobe_sequence.txt.
